In this notebook the structure of the CORD-19 metadata.csv file is explored to the end of arriving at an appropriate database structure.

# TOC

1. [Imports](#Imports)
1. [metadata.csv](#metadata_csv)
    1. [Columns](#Columns)
       1. [Single valued and unique](#Single_valued_and_unique)
       1. [Single valued](#Single_valued)
1. [Setting up a schema](#Setting_up_a_schema)
    1. [Main table](#Main_table)
    1. [Multivalued field tables](#Multivalued_field_tables)
       1. [pdf_json_files](#pdf_json_files)
       1. [pmc_json_files](#pmc_json_files)
       1. [sha](#sha)
       1. [authors](#authors)
    1. [Quick test](#Quick_test)

<a id='Imports'></a>
# Imports

In [1]:
import csv
import pandas as pd
from pprint import pprint as pp

In [2]:
DataDirectory = './cord-19_2020-06-01/'

<a id='metadata_csv'></a>
# metadata.csv

In [3]:
metadataFile = open(DataDirectory+'metadata.csv', 'r')
csvReader = csv.reader(metadataFile, delimiter=',', quotechar='"')

Header = next(csvReader)
pp(Header)

['cord_uid',
 'sha',
 'source_x',
 'title',
 'doi',
 'pmcid',
 'pubmed_id',
 'license',
 'abstract',
 'publish_time',
 'authors',
 'journal',
 'mag_id',
 'who_covidence_id',
 'arxiv_id',
 'pdf_json_files',
 'pmc_json_files',
 'url',
 's2_id']


<a id='Columns'></a>
## Columns

Starting from the readme provided with the data (metadata.readme) we can outline what each columns should cover and, importantly for databasing it, whether it is single value of potentially multi-valued.

'cord_uid' - This is a persistent identifier for the article within the CORD-19 ecosystem. __single valued__, __unique__.

'sha' - The hash of the article's PDF. There may be multiple PDFs associated to one article (supporting materials, ?preprints?). __multi-valued__.

'source_x' - The source of the article data. Sources covered include CZI (Chan-Zuckerberg Initiative), PMC (pubmed central), bioRxiv and medRxiv. Confirm sources from data. Should be __single valued__.

'title' - The article's title. __single valued__.

'doi' - The article's doi. __single valued__.

'pmcid' - The article's pubmed central id, if it has one. __single valued__, __unique__.


'pubmed_id' - The article's pubmed id. For distinction between this and the previous, see [here](https://publicaccess.nih.gov/include-pmcid-citations.htm#Difference). __single valued__, __unique__.

'license' - The license under which the data is being shared. Should be __single valued__.

'abstract' - The article's abstract. __single valued__.

'publish_time' - The date the article was published. __single valued__.

'authors' - The article's authors. Coverage questionable. __multi-valued__.

'journal' - The journal in which the article was published. If it was published in a journal. What happens with preprints? __single valued__.

'mag_id' - The article's microsoft academic graph id. __single valued__, __unique__.

'who_covidence_id' - The article's WHO #Covidence id. Should only be populated for CZI source articles. __single valued__, __unique__.

'arxiv_id' - The article's arXiv id. __single valued__, __unique__.

'pdf_json_files' - Relative path of the json(s) parsed from pdf file(s). __multi-valued__, __unique__.

'pmc_json_files' - Relative path of the json(s) parsed from pmc file(s). __multi-valued__, __unique__.

'url' - The article's URL. Should be __single valued__, __unique__.

's2_id' - The article's Semantic Scholar id. Can be resolved at https://api.semanticscholar.org/. Should be __single valued__, __unique__.


To explore the columns, it is probably easier to work with the csv via pandas.

In [4]:
metadataFrame = pd.read_csv(DataDirectory+'metadata.csv', low_memory=False)
metadataFrame

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
0,ug7v899j,d1aafb70c066a2068b02786f8929fd9c900897fb,PMC,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,PMC35282,11472636.0,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,NaN,NaN,NaN,document_parses/pdf_json/d1aafb70c066a2068b027...,document_parses/pmc_json/PMC35282.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
1,02tnwd4m,6b0567729c2143a66d737eb0a2f63f2dce2e5a7d,PMC,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,PMC59543,11667967.0,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/6b0567729c2143a66d737...,document_parses/pmc_json/PMC59543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
2,ejv2xln0,06ced00a5fc04215949aa72528f2eeaae1d58927,PMC,Surfactant protein-D and pulmonary host defense,10.1186/rr19,PMC59549,11667972.0,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/06ced00a5fc04215949aa...,document_parses/pmc_json/PMC59549.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3,2b73a28n,348055649b6b8cf2b9a376498df9bf41f7123605,PMC,Role of endothelin-1 in lung disease,10.1186/rr44,PMC59574,11686871.0,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/348055649b6b8cf2b9a37...,document_parses/pmc_json/PMC59574.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4,9785vg6d,5f48792a5fa08bed9f56016f4981ae2ca6031b32,PMC,Gene expression in epithelial cells in respons...,10.1186/rr61,PMC59580,11686888.0,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,NaN,NaN,NaN,document_parses/pdf_json/5f48792a5fa08bed9f560...,document_parses/pmc_json/PMC59580.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
5,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...,NaN
6,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",10.1093/emboj/21.9.2076,PMC125375,11980704.0,green-oa,CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/e3d0d482ebd9a8ba81c25...,document_parses/pmc_json/PMC125375.xml.json,http://europepmc.org/articles/pmc125375?pdf=re...,NaN
7,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein...,10.1093/emboj/20.14.3840,PMC125543,11447125.0,no-cc,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",EMBO J,NaN,NaN,NaN,document_parses/pdf_json/00b1d99e70f779eb4ede5...,document_parses/pmc_json/PMC125543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,NaN
8,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase revea...,10.1093/emboj/cdf327,PMC126080,12093723.0,green-oa,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/cf584e00f637cbd8f1bb3...,document_parses/pmc_json/PMC126080.xml.json,http://e

<a id='Single_valued_and_unique'></a>
### Single valued and unique

First start by making sure the columns that are supposed to be single valued and unique are actually so.

In [5]:
SingleValued_and_Unique = ['cord_uid', 'pmcid', 'pubmed_id', 'mag_id', 'who_covidence_id', 'arxiv_id', 'url', 's2_id']

In [6]:
for Column in SingleValued_and_Unique:
    print(Column, metadataFrame[Column].dropna().is_unique)

cord_uid False
pmcid True
pubmed_id False
mag_id True
who_covidence_id True
arxiv_id True
url False
s2_id False


All except mag_id, who_covidence_id and arxiv_id appear to be non-unique. Less than optimal...

Start by checking the cord_uid.

In [7]:
ValueCounts = metadataFrame['cord_uid'].value_counts()
ValueCounts[ValueCounts>1]

uym826bh    6
hgpgeel6    5
lhj7cd8t    5
30sjxtnk    5
3l4mntw2    4
q66wkdkd    4
7plpn6sn    4
4kgndu36    4
453q288g    4
lmo0ry4k    4
ftanlk8k    4
hkm8yspk    4
opwd03wp    4
9naohwpo    4
4wsueam1    4
tnctqc1a    4
q6px4r3j    4
v88f9e7m    4
hl8v7n66    4
mejio1m7    4
c9jg3bf3    4
kriro0t4    4
w0brwyby    4
ciqrsfpo    4
f8hgcngj    4
8xz0ddci    4
trzkb9ne    4
ux7bdk9x    4
3msk5ggc    4
klseuylw    4
           ..
05xedq1l    2
ziepfnpz    2
ucorpfw5    2
6donibpz    2
nn2an4up    2
ydq687u6    2
q5dhugfa    2
p370gjjn    2
28ygsbo1    2
3xe2jjl4    2
o6bkxn7l    2
ojtp90na    2
2tu707ng    2
eich19nx    2
7xb7hj9u    2
pauzgepm    2
bim76jna    2
h10o18ss    2
olveh2gf    2
ywkavncn    2
56lr1rlf    2
l0ggbztj    2
cb56np26    2
0irr4tnw    2
a6o88zlr    2
qqh7ayga    2
l0ongbag    2
0n5bxp93    2
ft9hclnz    2
fcd07krs    2
Name: cord_uid, Length: 781, dtype: int64

As this is supposed to the persistent identifier, looks like some duplicates happening. Close to 800 in fact.

So the easiest way to start with this is by checking to see if these are duplicates, and if they are just eliminating the double records.

In [8]:
metadataFrame[metadataFrame['cord_uid'].duplicated(keep=False)].sort_values(by=['cord_uid'])

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
27100,028avudf,NaN,Medline,Burden and prevalence of prognostic factors fo...,10.1007/s10654-020-00646-z,NaN,32425318.0,unk,The World Health Organization and European Cen...,2020-05-18,"Gémes, Katalin; Talbäck, Mats; Modig, Karin; A...",European journal of epidemiology,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1007/s10654-020-00646-z; ht...,218675407.0
60114,028avudf,NaN,WHO,Burden and prevalence of prognostic factors fo...,10.1007/s10654-020-00646-z,NaN,NaN,unk,The World Health Organization and European Cen...,2020,"Gémes, Katalin; Talbäck, Mats; Modig, Karin; A...",Eur J Epidemiol,NaN,#291472,NaN,NaN,NaN,https://doi.org/10.1007/s10654-020-00646-z,218675407.0
134087,028avudf,4e11d7fbd071bf098dddb9e5beca620fcdaacc39,Medline; PMC,Burden and prevalence of prognostic factors fo...,10.1007/s10654-020-00646-z,PMC7233678,32424571.0,cc-by,The World Health Organization and European Cen...,2020-05-18,"Gémes, Katalin; Talbäck, Mats; Modig, Karin; A...",Eur J Epidemiol,NaN,NaN,NaN,document_parses/pdf_json/4e11d7fbd071bf098dddb...,document_parses/pmc_json/PMC7233678.xml.json,https://doi.org/10.1007/s10654-020-00646-z; ht...,218689939.0
97788,04jbodmf,NaN,Medline; WHO,Epidemiologic characteristics of early cases w...,10.4178/epih.e2020007,NaN,32035431.0,unk,In about 20 days since the diagnosis of the fi...,2020,"Ki, Moran; Task Force for -nCo, V.",Epidemiology and health,NaN,#10145,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/32035431/;...,211072168.0
65413,04jbodmf,NaN,WHO,Epidemiologic characteristics of early cases w...,10.4178/epih.e2020007,NaN,NaN,unk,Since the first case of 2019 novel coronavirus...,2020,"Ki, Moran; nCoV, Task Force For",Epidemiol Health,NaN,#567,NaN,NaN,NaN,https://doi.org/10.4178/epih.e2020007,211072168.0
105132,052xdp69,9d9d15ee5ecd1555d091ef0a2a36485d8c810e49,Medline; PMC,"PERSONAS MAYORES, DEPENDENCIA Y VULNERABILIDAD...",10.1016/j.enfcli.2020.05.004,PMC7229944,32425485.0,no-cc,The current SARS-CoV-2 pandemic (COVID-19) is ...,2020-05-16,"Gálvez, Ana Mª Porcel; Romero, Bárbara Badanta...",Enferm Clin,NaN,NaN,NaN,document_parses/pdf_json/9d9d15ee5ecd1555d091e...,NaN,https://www.ncbi.nlm.nih.gov/pubmed/32425485/;...,218688637.0
61380,052xdp69,9d9d15ee5ecd1555d091ef0a2a36485d8c810e49,Elsevier,"PERSONAS MAYORES, DEPENDENCIA Y VULNERABILIDAD...",10.1016/j.enfcli.2020.05.004,NaN,NaN,els-covid,Resumen La pandemia actual por SARS-CoV-2 (COV...,2020-05-16,"Gálvez, Ana Mª Porcel; Romero, Bárbara Badanta...",NaN,NaN,NaN,NaN,document_parses/pdf_json/9d9d15ee5ecd1555d091e...,NaN,https://api.elsevier.com/content/article/pii/S...,218656984.0
61379,052xdp69,NaN,WHO,"Personas Mayores, Dependencia Y Vulnerabilidad...",10.1016/j.enfcli.2020.05.004,NaN,NaN,unk,The current SARS-CoV-2 pandemic (COVID-19) is ...,2020,"Gálvez, Ana Mª Porcel; Romero, Bárbara Badanta...",Enferm. clin. (Ed. impr.),NaN,#276713,NaN,NaN,NaN,https://doi.org/10.1016/j.enfcli.2020.05.004,218656984.0
51971,05my504t,NaN,Medline,COVID-19 and Paediatric Inflammatory Bowel Dis...,10.1097/mpg.0000000000002729,NaN,32235161.0,unk,INTRODUCTION With the current COVID-19 pandemi...,2020-03-31,"Turner, Dan; Huang, Ying; Martín-de-Carpi, Jav...",Journal of pediatric gastroenterology and nutr...,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1097/mpg.0000000000002729; ...,214772386.0
56347,05my504t,NaN,Medline,Corona Virus Disease 2019 and Paediatric Infla...,10.1097/mpg.0000000000002729,NaN,32443020.0,unk,INTRODUCTION With the current coronavirus dise...,2020-06-01,"Turner, Dan; Huang, Ying; Martín-de-Carpi, Jav...",Journal of pediatric gastroenterology and nutr...,NaN,NaN,NaN,NaN,NaN,https://doi.org/10.1097/mpg.0000000000002729; ...,218856260.0


It looks like this is primarily an issue of source.

For now just drop these records. A little under 800 will not have an impact within a corpus of 140k+

In [9]:
print('Rows before: {}'.format(metadataFrame.shape[0]))
metadataFrame = metadataFrame.drop_duplicates('cord_uid', keep=False) # drop all duplicates. keep='first' or keep='last' would keep the first/last
print('Rows after: {}'.format(metadataFrame.shape[0]))

Rows before: 140532
Rows after: 138751


In [10]:
metadataFrame['cord_uid'].dropna().is_unique

True

So that worked. cord_uid is now unique.

In [11]:
for Column in SingleValued_and_Unique:
    print(Column, metadataFrame[Column].dropna().is_unique)

cord_uid True
pmcid True
pubmed_id False
mag_id True
who_covidence_id True
arxiv_id True
url True
s2_id False


Also cleared up pmcid and url. Would have been nice if it cleared up a few of the others too though. Let's do a quick check of what discrepancies are left.

In [12]:
# For each column in SingleValued_and_Unique, this prints the values that appear in more than 1 record.
for Column in SingleValued_and_Unique:
    ValueCounts = metadataFrame[Column].value_counts()
    print(ValueCounts[ValueCounts>1])

Series([], Name: cord_uid, dtype: int64)
Series([], Name: pmcid, dtype: int64)
32419926.0    3
16575523.0    2
17098273.0    2
23842446.0    2
1655870.0     2
16115318.0    2
15313178.0    2
8546012.0     2
26499340.0    2
8545971.0     2
11918861.0    2
207058.0      2
16116432.0    2
16112641.0    2
16112593.0    2
1331160.0     2
1656597.0     2
16112259.0    2
9058755.0     2
16111992.0    2
15316497.0    2
26509109.0    2
1655802.0     2
9050994.0     2
26475793.0    2
18095180.0    2
16417929.0    2
2997998.0     2
32238613.0    2
16118891.0    2
             ..
12683406.0    2
25458561.0    2
16964581.0    2
17030494.0    2
16534565.0    2
6171237.0     2
17037549.0    2
2446421.0     2
6199888.0     2
17037602.0    2
16781410.0    2
12484662.0    2
17037504.0    2
17002511.0    2
25558126.0    2
12781012.0    2
196571.0      2
17037499.0    2
17037567.0    2
17922397.0    2
16963099.0    2
16582497.0    2
16781449.0    2
12377851.0    2
17052935.0    2
16824137.0    2
16940861.

__pubmed_id__
This has almost 7000 duplicates. Lets have a quick look.

In [13]:
# These get a bit complicated. Basically it starts by selecting the rows that have duplicate (non-null) pubmed_id

metadataFrame[metadataFrame['pubmed_id'].duplicated(keep=False) & metadataFrame['pubmed_id'].notnull()].sort_values(by=['pubmed_id'])

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
74183,crz30eba,NaN,Medline,Intestinal influenza: replication and characte...,NaN,NaN,23604.0,unk,NaN,1978,"Webster, R G; Yakhno, M; Hinshaw, V S; Bean, W...",Virology,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/23604/,44623236.0
137250,nl5kmmtd,3a2ff3a86c4b06fb35f0422034ce8c65f7ca0bd3,Elsevier; PMC,Intestinal influenza: Replication and characte...,10.1016/0042-6822(78)90247-7,PMC7131577,23604.0,els-covid,Abstract Influenza A viruses isolated from the...,1978-02-28,"Webster, Robert G.; Yakhno, Maya; Hinshaw, Vir...",Virology,NaN,NaN,NaN,document_parses/pdf_json/3a2ff3a86c4b06fb35f04...,NaN,https://api.elsevier.com/content/article/pii/0...,44623236.0
76159,4l35bv99,NaN,Medline,Resolution of the major poliovirus capsid poly...,NaN,NaN,27002.0,unk,NaN,1978,"Vrijsen, R; Wouters, M; Boeye, A",Virology,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/27002/,45109889.0
136565,m9j3pcj1,5610c53dce08ee3038eade1231d7d6d0fc6a8ad9,Elsevier; PMC,Resolution of the major poliovirus capsid poly...,10.1016/0042-6822(78)90093-4,PMC7131310,27002.0,els-covid,Abstract Using the pH gradient electrophoretic...,1978-05-15,"Vrijsen, R.; Wouters, M.; Boeye, A.",Virology,NaN,NaN,NaN,document_parses/pdf_json/5610c53dce08ee3038ead...,NaN,https://api.elsevier.com/content/article/pii/0...,45109889.0
78077,3e7gpq2z,NaN,Medline,Physico-chemical properties of mouse hepatitis...,NaN,NaN,30881.0,unk,Some properties of a strain of mouse hepatitis...,1978,"Hirano, N; Hino, S; Fujiwara, K",Microbiology and immunology,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/30881/,43253442.0
13229,smqmp1mm,8090cf4603341df5350674cd0ab3f9877f67f95e,PMC,Physico‐Chemical Properties of Mouse Hepatitis...,10.1111/j.1348-0421.1978.tb00384.x,PMC7168437,30881.0,no-cc,Some properties of a strain of mouse hepatitis...,2013-11-14,"Hirano, Norio; Hino, Shigeo; Fujiwara, Kosaku",Microbiol Immunol,NaN,NaN,NaN,document_parses/pdf_json/8090cf4603341df535067...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
125393,747jppsc,7747dfcb318bbbf7f9edebb69bc91c4b60f5e658,Elsevier; PMC,PLEOMORPHIC VIRUS-LIKE PARTICLES IN HUMAN FÆCES,10.1016/s0140-6736(75)91832-2,PMC7173216,48733.0,els-covid,Abstract Pleomorphic fringed particles bearing...,1975-05-10,"Mathan, M.; Swaminathan, S. P.; Mathan, V. I.;...",The Lancet,NaN,NaN,NaN,document_parses/pdf_json/7747dfcb318bbbf7f9ede...,NaN,https://www.sciencedirect.com/science/article/...,40081364.0
79613,u07xr5nt,NaN,Medline,Pleomorphic virus-like particles in human faeces.,NaN,NaN,48733.0,unk,Pleomorphic fringed particles bearing some res...,1975,"Mathan, M; Mathan, V I; Swaminathan, S P; Yesu...",Lancet,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/48733/,40081364.0
83782,100z5zox,NaN,Medline,Coronavirus propagated from patient with non-b...,NaN,NaN,53434.0,unk,A faecal suspension from a patient with gastro...,1975,"Caul, E O; Clarke, S K",Lancet,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/53434/,31630465.0
97482,h51cebpe,71d3406258973d7ea61a630bf16f5cfa04f78a13,Elsevier; PMC,CORONAVIRUS PROPAGATED FROM PATIENT WITH NON-B...,10.1016/s0140-6736(75)90363-3,PMC7135454,53434.0,els-covid,Abstract A fæcal suspension from a patient wit...,1975-11-15,"Caul, E. O.; Clarke, S.K.R.",The Lancet,NaN,NaN,NaN,document_parses/pdf_json/71d3406258973d7ea61a6...,NaN,https://api.elsevier.com/content/article/pii/S...,31630465.0


This looks like a case where one has a pmcid and the other does not.

I would like to keep the record that has a document parse.

What I am going to do is a bit tricksy.

I sort the dataframe by the parsefile columns. Then I use the duplicated funciton to keep only the first occuring row for each pubmed_id.

In [14]:
print('Rows before: {}'.format(metadataFrame.shape[0]))
metadataFrame = metadataFrame[~(metadataFrame.sort_values(by=['pdf_json_files', 'pmc_json_files']).duplicated('pubmed_id', keep='last') & metadataFrame['pubmed_id'].notnull())]
print('Rows after: {}'.format(metadataFrame.shape[0]))

Rows before: 138751
Rows after: 131846


It eliminated about 7000 rows, matching what number of duplicates we found above.

In [15]:
metadataFrame['pubmed_id'].dropna().is_unique

True

And, indeed, pubmed_id is now unique.

Now turning to __s2_id__, which recalling from above had over 13k duplicates.

In [16]:
metadataFrame[metadataFrame['s2_id'].duplicated(keep=False) & metadataFrame['s2_id'].notnull()].sort_values(by=['s2_id'])

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
74070,mo0bhc4s,NaN,Medline,Modeling the worldwide spread of pandemic infl...,NaN,NaN,17253899.0,unk,BACKGROUND The highly pathogenic H5N1 avian in...,2007,"Colizza, Vittoria; Barrat, Alain; Barthelemy, ...",PLoS medicine,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/17253899/,72538.0
62618,7d7ewkib,adae39fea7d473e279cfffb02a95617e2bd5d65c,ArXiv,Modeling the Worldwide Spread of Pandemic Infl...,10.1371/journal.pmed.0040013,NaN,NaN,arxiv,We present a study of the worldwide spread of ...,2007-01-24,"Colizza, Vittoria; Barrat, Alain; Barthelemy, ...",NaN,NaN,NaN,q-bio/0701038,document_parses/pdf_json/adae39fea7d473e279cff...,NaN,https://arxiv.org/pdf/q-bio/0701038v1.pdf,72538.0
134234,kllv27bl,aa7094234bc71c19d615cc130ca8c2ef98f543a9,Elsevier; Medline; PMC,Evolution of Chinese airport network,10.1016/j.physa.2010.05.042,PMC7127146,32288080.0,els-covid,Abstract With the rapid development of the eco...,2010-09-15,"Zhang, Jun; Cao, Xian-Bin; Du, Wen-Bo; Cai, Ka...",Physica A: Statistical Mechanics and its Appli...,NaN,NaN,NaN,document_parses/pdf_json/aa7094234bc71c19d615c...,document_parses/pmc_json/PMC7127146.xml.json,https://www.sciencedirect.com/science/article/...,75478.0
85508,34qmk3jx,c1408f066c54987b04e0b997c6b38197f7ad6e3f,ArXiv,Evolution of Chinese airport network,NaN,NaN,NaN,arxiv,With the rapid development of economy and the ...,2011-01-03,"Zhang, Jun; Cao, Xian-Bin; Du, Wen-Bo; Cai, Ka...",NaN,NaN,NaN,1101.0656,document_parses/pdf_json/c1408f066c54987b04e0b...,NaN,https://arxiv.org/pdf/1101.0656v1.pdf,75478.0
78579,t92nyi3g,NaN,Medline,Semiparametric Relative-risk Regression for In...,NaN,NaN,26146425.0,unk,This paper introduces semiparametric relative-...,2015,"Kenah, Eben",Journal of the American Statistical Association,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pubmed/26146425/,357136.0
64327,auasueaw,NaN,ArXiv,Semiparametric Relative-risk Regression for In...,10.1080/01621459.2014.896807,NaN,NaN,arxiv,This paper introduces semiparametric relative-...,2012-10-17,"Kenah, Eben",NaN,NaN,NaN,1210.4630,NaN,NaN,https://arxiv.org/pdf/1210.4630v1.pdf,357136.0
85421,lo3m6csl,466dc1e5c227ca49cd0fc535dd402f5049b9161b,ArXiv,Inverse Folding of RNA Pseudoknot Structures,NaN,NaN,NaN,arxiv,Background: RNA exhibits a variety of structur...,2010-03-10,"Gao, James Z.M.; Li, Linda Y.M.; Reidys, Chris...",NaN,NaN,NaN,1003.2015,document_parses/pdf_json/466dc1e5c227ca49cd0fc...,NaN,https://arxiv.org/pdf/1003.2015v1.pdf,389081.0
110473,1lojd0xa,a3778f543ef74ff3a0dd90250221b77ee38d7533,Medline; PMC,Inverse folding of RNA pseudoknot structures,10.1186/1748-7188-5-27,PMC2909241,20573197.0,cc-by,BACKGROUND: RNA exhibits a variety of structur...,2010-06-23,"Gao, James ZM; Li, Linda YM; Reidys, Christian M",Algorithms Mol Biol,NaN,NaN,NaN,document_parses/pdf_json/a3778f543ef74ff3a0dd9...,document_parses/pmc_json/PMC2909241.xml.json,https://doi.org/10.1186/1748-7188-5-27; https:...,389081.0
85420,kto778l5,081fce9252d6f843d2329db65ae188d41bf4c30f,ArXiv,Inverse folding of RNA pseudoknot structures,NaN,NaN,NaN,arxiv,Background: RNA exhibits a variety of structur...,2009-05-06,"Gao, James Z. M.; Li, Linda Y. M.; Reidys, Chr...",NaN,NaN,NaN,0905.0733,document_parses/pdf_json/081fce9252d6f843d2329...,NaN,https://arxiv.org/pdf/0905.0733v2.pdf,389081.0
95650,5qp7vk0c,65078532d15de7dbfa1931a5e6e6eba317cee5e3,ArXiv,Eliciting Disease Data from Wikipedia Articles,NaN,NaN,NaN,arxiv,Traditional disease surveillance systems suffe...,2015-04-02,"Fairchild, Geoffrey; Silva, Lalindra De; Valle...",NaN,NaN,NaN,1504.00657,document_parses/pdf_json/65078532d15de7dbfa193...,NaN,https://arxiv.org/pdf/1504.00657v3.pdf,1254084.0


The discrepancies here seem to arise from cases where Semantic Scholar has (seemingly correctly) grouped a published paper with its ArXiv version but the abstract differed between the two versions. This is really some tricky territory. My inclination at the moment is to leave this as is, and circle back to it if necessary.

Lets finish by just checking again all the columns that should be single valued and unique.

In [17]:
for Column in SingleValued_and_Unique:
    print(Column, metadataFrame[Column].dropna().is_unique)

cord_uid True
pmcid True
pubmed_id True
mag_id True
who_covidence_id True
arxiv_id True
url True
s2_id False


So other than s2_id all are now unique.

In [18]:
metadataFrame.shape[0]

131846

And have arrived at 131,846 records.

<a id='Single_valued'></a>
### Single valued

Based on investigations to the this point, it looks like they are denoting multiple values within a column via a semicolon (;). Thus it is likely best to quickly check to make sure the columns we inferred to be inferred to be single valued are actually so.

In [19]:
SingleValued = ['title', 'doi', 'license', 'publish_time']

In [20]:
for Column in SingleValued:
    print(Column, metadataFrame[Column].str.contains(';').fillna(False).sum())

title 575
doi 25
license 0
publish_time 0


license and publish_time are single valued.

doi has a few cases that are not, will have a look but likely ok to just leave as a single valued variable and deal with the corner case if we end up working with doi.

title need to be investigated.

In [21]:
metadataFrame[metadataFrame['doi'].str.contains(';').fillna(False)]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
8394,no7mnxtj,a1bf6ada1b47e286db751f32fb6d74a6a41328b4,PMC,Generation of an immortal differentiated lung ...,10.1290/1071-2690(2000)036<0374:goaidl>2.0.co;2,PMC7101677,10949996.0,no-cc,This paper describes a new fully differentiate...,2000,"deMello, Daphne E.; Mahmoud, Sohir; Padfield, ...",In Vitro Cell Dev Biol Anim,NaN,NaN,NaN,document_parses/pdf_json/a1bf6ada1b47e286db751...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12259,djxoiytq,64066db1148876bb85f1c65e2acfab11dfd89c11,PMC,Molecular mimicry and multiple sclerosis: Dege...,10.1002/1531-8249(199905)45:5<559::aid-ana3>3....,PMC7159663,10319877.0,no-cc,Various mechanisms have been proposed for the ...,2001-06-01,"Gran, Bruno; Hemmer, Bernhard; Vergelli, Marco...",Ann Neurol,NaN,NaN,NaN,document_parses/pdf_json/64066db1148876bb85f1c...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12262,lreguu45,01207321329715e944d0ae2cba853b919340ad91,PMC,Increased serum levels of cartilage oligomeric...,10.1002/1529-0131(199803)41:3<544::aid-art21>3...,PMC7159671,9506583.0,no-cc,OBJECTIVE: To investigate the utility of serum...,2004-05-28,"Vingsbo‐Lundberg, Carina; Saxne, Tore; Olsson,...",Arthritis Rheum,NaN,NaN,NaN,document_parses/pdf_json/01207321329715e944d0a...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12422,23cdi61w,11c686b3de7f984cb5e82860014a23050d7cf740,PMC,Electron microscopic observations on structure...,10.1002/1097-0142(197106)27:6<1449::aid-cncr28...,PMC7162171,5282620.0,no-cc,Human tumors of mesenchymal origin contain cyt...,2006-06-27,"Györkey, Ferenc; Sinkovics, Joseph G.; Györkey...",Cancer,NaN,NaN,NaN,document_parses/pdf_json/11c686b3de7f984cb5e82...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12435,2cv7cs48,a0e60329e1527b513e6bfcc6f5da46c7bbf19bf8,PMC,Tubular complexes of endoplasmic reticulum in ...,10.1002/1097-0142(19820415)49:8<1629::aid-cncr...,PMC7162261,6978174.0,no-cc,Distinctive intracytoplasmic tubular complexes...,2006-06-28,"Chu, Helen; Foucar, Kathy; Barlogie, Barthel; ...",Cancer,NaN,NaN,NaN,document_parses/pdf_json/a0e60329e1527b513e6bf...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12444,cuy6dihq,279b4d3afbe2142f1ca72d7536b78b042ce35529,PMC,Kaposi's sarcoma in renal transplant recipient...,10.1002/1097-0142(19840115)53:2<258::aid-cncr2...,PMC7162315,6418373.0,no-cc,Tissues from four cases of Kaposi's sarcoma de...,2006-06-29,"Akhtar, Mohammed; Bunuan, Hernando; Ali, Moham...",Cancer,NaN,NaN,NaN,document_parses/pdf_json/279b4d3afbe2142f1ca72...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12451,khxb15wp,c5bcf6c3957245b4a27e4cadb72c7f1181fc1a32,PMC,Morphologic study of virus‐like particles in a...,10.1002/1097-0142(197604)37:4<1718::aid-cncr28...,PMC7162349,177173.0,no-cc,"Virus‐like particles, grouped in clusters not ...",2006-06-28,"Foa, C.; Foa, J.; Carcassonne, Y.",Cancer,NaN,NaN,NaN,document_parses/pdf_json/c5bcf6c3957245b4a27e4...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12456,0mw8x66n,e374c4e801f0bd2473b833d5aa9763fa9a1fcec0,PMC,Introduction to virus‐caused cancers,10.1002/1097-0142(197410)34:8+<1347::aid-cncr2...,PMC7162371,4422736.0,no-cc,Members of four different groups of animal vir...,2006-06-28,"Temin, Howard M.",Cancer,NaN,NaN,NaN,document_parses/pdf_json/e374c4e801f0bd2473b83...,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...,NaN
12563,2rdtj01s,dc0c47c12c9fddc79adc3c14760cae6dd914eaa2,PMC,The role of ecological theory and practice in ...,10.1890/1540-9295(2006)4[533:troeta]2.0.co;2,PMC7164082,32313512.0,no-cc,The fight against global poverty has gained mo...,2006-12-01,"DeClerck, Fabrice; Ingram, Jane C.; Rumbaitis ...",Front Ecol Environ,NaN,NaN,NaN,document_parses/pdf_json/dc0c47c12c9fddc79adc3...,document_parses/pmc_json/PMC7164082.xml.json,https://www.ncbi.nlm.nih.gov

So the __doi__ cases containing ';' just look like mis-parsings. Will assume doi is single valued and come back to this matter if we end up having to use doi for something.

Turning to __title__:

In [22]:
metadataFrame[metadataFrame['title'].str.contains(';').fillna(False)]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url,s2_id
1840,fkqdrwer,5e0504a1c581c36bbf2c2c97439b87b9834b108b,PMC,Induction of Interferon-Stimulated Genes on th...,10.1371/journal.pone.0064868,PMC3664578,23724103.0,cc-by,Epstein-Barr virus (EBV) is an oncogenic virus...,2013-05-27,"Smith, Nikki; Tierney, Rosemary; Wei, Wenbin; ...",PLoS One,NaN,NaN,NaN,document_parses/pdf_json/5e0504a1c581c36bbf2c2...,document_parses/pmc_json/PMC3664578.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,NaN
2271,s4qki9l5,f4c43e4ae49ca69dbac32620bd0a73ecbb683b91,PMC,Exploring the Innate Immunological Response of...,10.1155/2014/913632,PMC4129158,25170519.0,cc-by,The common marmoset (Callithrix jacchus) is in...,2014-07-22,"Nelson, M.; Loveday, M.",J Immunol Res,NaN,NaN,NaN,document_parses/pdf_json/f4c43e4ae49ca69dbac32...,document_parses/pmc_json/PMC4129158.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN
2509,l2dc9x12,d760a0c49650a6c2a3d822bab4417057e6be1cd1,PMC,"Correction: Xie, H.; et al. 3D QSAR Studies, P...",10.3390/ijms16035072,PMC4394465,25751723.0,cc-by,NaN,2015-03-05,"Xie, Huiding; Qiu, Kaixiong; Xie, Xiaoguang",Int J Mol Sci,NaN,NaN,NaN,document_parses/pdf_json/d760a0c49650a6c2a3d82...,document_parses/pmc_json/PMC4394465.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN
2615,9gftenfz,f4ac2b1e9b9523671d7c38c27a19f1eec2e15d75; 6044...,PMC,Meta-genomic analysis of toilet waste from lon...,10.1038/srep11444,PMC4498435,26161690.0,cc-by,Human populations worldwide are increasingly c...,2015-07-10,"Nordahl Petersen, Thomas; Rasmussen, Simon; Ha...",Sci Rep,NaN,NaN,NaN,document_parses/pdf_json/f4ac2b1e9b9523671d7c3...,document_parses/pmc_json/PMC4498435.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN
2880,k81qwzof,a7154cf3907686df4aab059a0be568e0dacb44cb,PMC,Surfactant Protein C-associated interstitial l...,10.1186/s13052-016-0235-x,PMC4772310,26925580.0,cc-by,BACKGROUND: Monoallelic mutations of the Surfa...,2016-02-29,"Salerno, Teresa; Peca, Donatella; Menchini, La...",Ital J Pediatr,NaN,NaN,NaN,document_parses/pdf_json/a7154cf3907686df4aab0...,document_parses/pmc_json/PMC4772310.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,NaN
3515,n7ev5osc,7018967c7ebc76e4c6d3decf7e836957ff427602,PMC,Correlation of central venous pressure with ve...,10.1016/j.tjem.2016.09.006,PMC5357094,28345066.0,cc-by-nc-nd,OBJECTIVE: This study was conducted to assess ...,2016-11-20,"Rahim-Taleghani, Sima; Fatemi, Alireza; Alavi ...",Turk J Emerg Med,NaN,NaN,NaN,document_parses/pdf_json/7018967c7ebc76e4c6d3d...,document_parses/pmc_json/PMC5357094.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
3641,muozucj7,NaN,PMC,Authors’ correction for Euro Surveill. 2017;22...,10.2807/1560-7917.es.2017.22.19.30531,PMC5476985,28537549.0,cc-by,NaN,2017-05-11,NaN,Euro Surveill,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,NaN
4377,eoz1l5fi,041e50456e5c62980ff8bb61d83283f58f09ef6d,PMC,Award Winners and Abstracts of the 32nd Annual...,10.1002/pro.3513,PMC6247239,30461082.0,bronze-oa,NaN,2018-11-01,NaN,Protein Science,NaN,NaN,NaN,document_parses/pdf_json/041e50456e5c62980ff8b...,NaN,https://onlinelibrary.wiley.com/doi/pdfdirect/...,NaN
4756,q10zmids,3061f05203159384dfbb2fd9b1d9a1ca7b98c8a6,PMC,Iranian Emergency Medical Service Response in ...,10.22114/ajem.v0i0.121,PMC6548112,31172124.0,cc-by-nc,INTRODUCTION: The earthquake is one of the mos...,2019-01-14,"Saberian, Peyman; Kolivand, Pir-Hossein; Hasan...",Adv J Emerg Med,NaN,NaN,NaN,document_parses/pdf_json/3061f05203159384dfbb2...,document_parses/pmc_json/PMC6548112.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6...,NaN
5456,5fk3clav,NaN,PMC,Erratum for Euro Surveill. 2020;25(5),10.2807/1560-7917.es.2020.25.6.2002132,PMC7029446,32070469.0,cc-by,NaN,2020-02-13,NaN,Euro Surveill,NaN,NaN,NaN,NaN,NaN,https://www.ncbi.n

Lets make these more readable.

In [23]:
for record in metadataFrame[metadataFrame['title'].str.contains(';').fillna(False)]['title']:
    pp(record)

('Induction of Interferon-Stimulated Genes on the IL-4 Response Axis by '
 'Epstein-Barr Virus Infected Human B Cells; Relevance to Cellular '
 'Transformation')
('Exploring the Innate Immunological Response of an Alternative Nonhuman '
 'Primate Model of Infectious Disease; the Common Marmoset')
('Correction: Xie, H.; et al. 3D QSAR Studies, Pharmacophore Modeling and '
 'Virtual Screening on a Series of Steroidal Aromatase Inhibitors. Int. J. '
 'Mol. Sci. 2014, 15, 20927–20947')
('Meta-genomic analysis of toilet waste from long distance flights; a step '
 'towards global surveillance of infectious diseases and antimicrobial '
 'resistance')
('Surfactant Protein C-associated interstitial lung disease; three different '
 'phenotypes of the same SFTPC mutation')
('Correlation of central venous pressure with venous blood gas analysis '
 'parameters; a diagnostic study')
'Authors’ correction for Euro Surveill. 2017;22(11)'
('Award Winners and Abstracts of the 32nd Annual Symposium of The

It looks like most of these are cases where a semicolon (';') is in the place of a colon (':').

If course generalizing from checking a small number can be dangerous, but for now just replace ';' with ':'.

In [24]:
metadataFrame['title'] = metadataFrame['title'].str.replace(';',':')

Just to check them all again:

In [25]:
for Column in SingleValued:
    print(Column, metadataFrame[Column].str.contains(';').fillna(False).sum())

title 0
doi 25
license 0
publish_time 0


This is something we can live with.

<a id='Setting_up_a_schema'></a>
# Setting up a schema

Recalling the fields covered above:

__single valued__, __unique__
* 'cord_uid'
* 'pmcid'
* 'pubmed_id'
* 'mag_id'
* 'who_covidence_id'
* 'arxiv_id'
* 'url'

__single valued__
* 'source_x'
* 'title'
* 'doi'
* 'license'
* 'abstract'
* 'publish_time'
* 'journal'
* 's2_id'

__multi-valued__, __unique__
* 'pdf_json_files'
* 'pmc_json_files'

__multi-valued__
* 'sha'
* 'authors'

The strategy here is to create a main table that stores all the single valued fields, and then build four more that cover the multi-valued fields.

cord_uid will be the primary index across all.

In [26]:
import sqlite3 # For now just going to populate in sqlite. Will probably put down to disk better later in MySQL.

In [27]:
conn = sqlite3.connect('CORD_metadata.db')
cur = conn.cursor()

<a id='Main_table'></a>
## Main table

In [28]:
# Organize the data a bit in a dict
columnData = {}
columnData['cord_uid'] = {'SQLName': 'cord_uid', 'SQLType': 'char(8) NOT NULL UNIQUE', 'PythonType': 'str'}
columnData['pmcid'] = {'SQLName': 'pmcid', 'SQLType': 'char(9) UNIQUE', 'PythonType': 'str'}
columnData['pubmed_id'] = {'SQLName': 'pubmed_id', 'SQLType': 'number(8,0) UNIQUE', 'PythonType': 'int'}
columnData['mag_id'] = {'SQLName': 'mag_id', 'SQLType': 'number(10,0) UNIQUE', 'PythonType': 'int'}
columnData['who_covidence_id'] = {'SQLName': 'who_covidence_id', 'SQLType': 'char(12) UNIQUE', 'PythonType': 'str'}
columnData['arxiv_id'] = {'SQLName': 'arxiv_id', 'SQLType': 'text UNIQUE', 'PythonType': 'str'}
columnData['url'] = {'SQLName': 'url', 'SQLType': 'text UNIQUE', 'PythonType': 'str'}
columnData['source_x'] = {'SQLName': 'source_x', 'SQLType': 'text', 'PythonType': 'str'}
columnData['title'] = {'SQLName': 'title', 'SQLType': 'text', 'PythonType': 'str'}
columnData['doi'] = {'SQLName': 'doi', 'SQLType': 'text', 'PythonType': 'str'}
columnData['license'] = {'SQLName': 'license', 'SQLType': 'text', 'PythonType': 'str'}
columnData['abstract'] = {'SQLName': 'abstract', 'SQLType': 'text', 'PythonType': 'str'}
columnData['publish_time'] = {'SQLName': 'publish_time', 'SQLType': 'DATE', 'PythonType': 'str'}
columnData['journal'] = {'SQLName': 'journal', 'SQLType': 'text', 'PythonType': 'str'}
columnData['s2_id'] = {'SQLName': 's2_id', 'SQLType': 'number(9,0)', 'PythonType': 'int'}

The SQL types were arrived at by inspection. It is likely that some can be improved but will circle back as necessary.

In [29]:
TableName = 'main'
cur.execute('DROP TABLE IF EXISTS {}'.format(TableName)) # always start fresh.
CreateTableQuery = 'CREATE TABLE {}('.format(TableName)
for Column in metadataFrame.columns:
    if Column in columnData:
        CreateTableQuery += '{SQLName} {SQLType}, '.format(**columnData[Column])
CreateTableQuery = CreateTableQuery[:-2]+')' # hate this [:-2] construction to eliminate the last ','. Could be impoved with a join on a list.
#print(CreateTableQuery)
cur.execute(CreateTableQuery)

In [30]:
# This is work around for correctly inserting NULL.
def FormatNULL(Value, pythonType):
    if pd.isnull(Value):
        return('NULL')
    else:
        if pythonType == 'str':
            return('"{}"'.format(Value.replace('"', "'")))
        else:
            return(str(Value))

In [31]:
# This one can be used for testing. Will exceed IO rate if run.
def INSERT_from_df_Single(_dataframe, _tableName, _columnData, _cursor):
    InsertQueryStub = 'INSERT INTO {} ({}) VALUES '.format(_tableName, ', '.join([col for col in _dataframe.columns if col in _columnData]))
    for Record in _dataframe.to_dict('records'):
        InsertQuery = InsertQueryStub + '({})'.format(', '.join([FormatNULL(Record[col], _columnData[col]['PythonType']) for col in _dataframe.columns if col in _columnData]))
        #print(InsertQuery)
        _cursor.execute(InsertQuery)

In [32]:
# This should be used to actually do the insert.
# Have updated this to work from a list and print when batchsize is hit.
def INSERT_from_df_Mass(_dataframe, _tableName, _columnData, _cursor, _batchSize):
    MassInsertQueryStub = 'INSERT INTO {} ({}) VALUES '.format(_tableName, ', '.join([col for col in _dataframe.columns if col in _columnData]))

    MassInsertValues = []
    for Record in _dataframe.to_dict('records'):
        MassInsertValues.append('({})'.format(', '.join([FormatNULL(Record[col], _columnData[col]['PythonType']) for col in _dataframe.columns if col in _columnData])))

        if len(MassInsertValues) >= _batchSize:
            MassInsertQuery = MassInsertQueryStub + ', '.join(MassInsertValues[:_batchSize])
            _cursor.execute(MassInsertQuery)
            del MassInsertValues[:_batchSize]

    MassInsertQuery = MassInsertQueryStub + ', '.join(MassInsertValues[:_batchSize])
    _cursor.execute(MassInsertQuery)

In [33]:
INSERT_from_df_Mass(metadataFrame, TableName, columnData, cur, 500) # Batches of 500

AttributeError: 'float' object has no attribute 'replace'

In [ ]:
cur.execute('SELECT COUNT(*) FROM {}'.format(TableName))
pp(cur.fetchone())

The right number of records.

In [ ]:
cur.execute('SELECT * FROM {} WHERE publish_time BETWEEN "2001-01-01" AND "2002-01-01"'.format(TableName))
pp(cur.fetchone())

That all lines up quite nicely.

<a id='Multivalued_field_tables'></a>
## Multivalued field tables

<a id='pdf_json_files'></a>
### pdf_json_files

In [ ]:
columnData = {}
columnData['cord_uid'] = {'SQLName': 'cord_uid', 'SQLType': 'char(8) NOT NULL', 'PythonType': 'str'}
columnData['pdf_json_files'] = {'SQLName': 'pdf_json_files', 'SQLType': 'text', 'PythonType': 'str'}

In [ ]:
MainIndex = 'cord_uid'
MultiValuedVariable = 'pdf_json_files'
TableName = MultiValuedVariable
cur.execute('DROP TABLE IF EXISTS {}'.format(TableName))
CreateTableQuery = 'CREATE TABLE {}({} {}, {} {})'.format(TableName, MainIndex, columnData[MainIndex]['SQLType'], MultiValuedVariable, columnData[MultiValuedVariable]['SQLType'])
#print(CreateTableQuery)
cur.execute(CreateTableQuery)

In [ ]:
# This is work around for correctly inserting NULL.
def FormatNULL(Value, pythonType):
    if pd.isnull(Value):
        return('NULL')
    else:
        if pythonType == 'str':
            return('"{}"'.format(Value.replace('"', "'")))
        else:
            return(str(Value))

In [ ]:
def INSERT_from_df_MultiValue_Mass(_dataframe, _tableName, _mainIndex, _multiValueColumn, _columnData, _cursor, _batchSize):
    MassInsertQueryStub = 'INSERT INTO {} ({}) VALUES '.format(_tableName, ', '.join([_mainIndex, _multiValueColumn]))

    MassInsertValues = []
    for Record in _dataframe.to_dict('records'):

        if pd.isnull(Record[_multiValueColumn]):
            MassInsertValues.append('({})'.format(', '.join([FormatNULL(Record[col], _columnData[col]['PythonType']) for col in [_mainIndex, _multiValueColumn]])))
        else:
            for Value in Record[_multiValueColumn].split(';'):
                MassInsertValues.append('({}, {})'.format(FormatNULL(Record[_mainIndex], _columnData[_mainIndex]['PythonType']), FormatNULL(Value.strip(), _columnData[_multiValueColumn]['PythonType'])))

        if len(MassInsertValues) >= _batchSize:
            MassInsertQuery = MassInsertQueryStub + ', '.join(MassInsertValues[:_batchSize])
            _cursor.execute(MassInsertQuery)
            del MassInsertValues[:_batchSize]

    MassInsertQuery = MassInsertQueryStub + ', '.join(MassInsertValues[:_batchSize])
    _cursor.execute(MassInsertQuery)

In [ ]:
INSERT_from_df_MultiValue_Mass(metadataFrame, TableName, MainIndex, MultiValuedVariable, columnData, cur, 500)

<a id='pmc_json_files'></a>
### pmc_json_files

In [ ]:
columnData = {}
columnData['cord_uid'] = {'SQLName': 'cord_uid', 'SQLType': 'char(8) NOT NULL', 'PythonType': 'str'}
columnData['pmc_json_files'] = {'SQLName': 'pmc_json_files', 'SQLType': 'text', 'PythonType': 'str'}

In [ ]:
MainIndex = 'cord_uid'
MultiValuedVariable = 'pmc_json_files'
TableName = MultiValuedVariable
cur.execute('DROP TABLE IF EXISTS {}'.format(TableName))
CreateTableQuery = 'CREATE TABLE {}({} {}, {} {})'.format(TableName, MainIndex, columnData[MainIndex]['SQLType'], MultiValuedVariable, columnData[MultiValuedVariable]['SQLType'])
#print(CreateTableQuery)
cur.execute(CreateTableQuery)

In [ ]:
INSERT_from_df_MultiValue_Mass(metadataFrame, TableName, MainIndex, MultiValuedVariable, columnData, cur, 500)

<a id='sha'></a>
### sha

In [ ]:
columnData = {}
columnData['cord_uid'] = {'SQLName': 'cord_uid', 'SQLType': 'char(8) NOT NULL', 'PythonType': 'str'}
columnData['sha'] = {'SQLName': 'sha', 'SQLType': 'text', 'PythonType': 'str'}

In [ ]:
MainIndex = 'cord_uid'
MultiValuedVariable = 'sha'
TableName = MultiValuedVariable
cur.execute('DROP TABLE IF EXISTS {}'.format(TableName))
CreateTableQuery = 'CREATE TABLE {}({} {}, {} {})'.format(TableName, MainIndex, columnData[MainIndex]['SQLType'], MultiValuedVariable, columnData[MultiValuedVariable]['SQLType'])
#print(CreateTableQuery)
cur.execute(CreateTableQuery)

In [ ]:
INSERT_from_df_MultiValue_Mass(metadataFrame, TableName, MainIndex, MultiValuedVariable, columnData, cur, 500)

<a id='authors'></a>
### authors

In [ ]:
columnData = {}
columnData['cord_uid'] = {'SQLName': 'cord_uid', 'SQLType': 'char(8) NOT NULL', 'PythonType': 'str'}
columnData['authors'] = {'SQLName': 'authors', 'SQLType': 'text', 'PythonType': 'str'}

In [ ]:
MainIndex = 'cord_uid'
MultiValuedVariable = 'authors'
TableName = MultiValuedVariable
cur.execute('DROP TABLE IF EXISTS {}'.format(TableName))
CreateTableQuery = 'CREATE TABLE {}({} {}, {} {})'.format(TableName, MainIndex, columnData[MainIndex]['SQLType'], MultiValuedVariable, columnData[MultiValuedVariable]['SQLType'])
#print(CreateTableQuery)
cur.execute(CreateTableQuery)

In [ ]:
INSERT_from_df_MultiValue_Mass(metadataFrame, TableName, MainIndex, MultiValuedVariable, columnData, cur, 500)

Those are all loaded quite well.

If it ends up necessary to go a step further it would be best to streamline this with a tool like SQLAlchemy.

<a id='Quick_test'></a>
## Quick test

In [ ]:
cur.execute('SELECT * FROM {} WHERE publish_time BETWEEN "2010-01-01" AND "2011-01-01"'.format('main'))
Record = cur.fetchone()
pp(Record)

In [ ]:
MultiValued = ['pdf_json_files', 'pmc_json_files', 'sha', 'authors']

for Table in MultiValued:
    cur.execute('SELECT {} FROM {} WHERE cord_uid IS "{}"'.format(Table, Table, Record[0]))
    pp(cur.fetchall())

So this appears to all be in order.